# S3

In [ ]:
import boto3
import zipfile
import os
import shutil

## Get From S3

In [ ]:
# S3 Setup
s3 = boto3.client("s3")
bucket_name = "ms-thesis-sagemaker"  # Replace with your S3 bucket
s3_file_path = "mbert_bpe_hf_dataset.zip"  # Replace with the file name in S3
local_zip_path = "/home/ec2-user/SageMaker/ms-thesis/model-variants/data/mbert_bpe_hf_dataset.zip"  # Where to save in SageMaker

# Download the ZIP file from S3
s3.download_file(bucket_name, s3_file_path, local_zip_path)
print("ZIP file downloaded from S3 successfully!")

ZIP file downloaded from S3 successfully!


In [ ]:
extract_path = "/home/ec2-user/SageMaker/ms-thesis/model-variants/data"  # Where to extract

# Unzip the file
with zipfile.ZipFile(local_zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("ZIP file extracted successfully!")

ZIP file extracted successfully!


In [15]:
os.remove(local_zip_path)
print("ZIP file deleted to free space.")

ZIP file deleted to free space.


## Move to S3

In [ ]:
# function to zip and upload
def zip_and_upload_to_s3(source_folder):
    bucket_name = "ms-thesis-sagemaker"

    zip_file = f"{source_folder}.zip"
    s3_key = f"uploads/{zip_file}"

    # Create a zip archive
    shutil.make_archive(zip_file.replace(".zip", ""), 'zip', source_folder)
    print(f"Zipped {source_folder} -> {zip_file}")

    # Upload to S3
    s3_client = boto3.client("s3")
    s3_client.upload_file(zip_file, bucket_name, s3_key)
    print(f"Uploaded to S3: s3://{bucket_name}/{s3_key}")

    # delete zip file
    os.remove(zip_file)
    print("ZIP file deleted to free space.")

    return f"s3://{bucket_name}/{s3_key}"

In [ ]:
# zip logs
zip_and_upload_to_s3("logs")

In [ ]:
# zip results
zip_and_upload_to_s3("model-variants/results")

In [ ]:
# zip results
zip_and_upload_to_s3("model-variants/models")

# Importing all the important libraries

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install transformers sentence_transformers sentencepiece peft datasets bert_score sacrebleu

In [ ]:
!conda update -n base -c conda-forge conda -y

In [ ]:
!conda install conda-forge::rouge-score -y

In [ ]:
import pandas as pd
import sentencepiece as spm
import torch.nn as nn
from utils.dataframe import (
    load_gen_df, save_tmp_df, load_tmp_df, load_models_df,
    save_model_variants_df, load_model_variants_df,
    save_model_variants_hf, load_model_variants_hf,
    save_model_variants_gen_df, load_model_variants_gen_df,
    convert_to_hf,
)
from utils.gpu import get_device
from utils.common import (
    generate_masked_predictions_hf, generate_mt5_predictions,
    compute_metrics_hf,
    convert_to_mean_scores_df,
    get_fine_tuned_model, get_embedded_fine_tuned_model,
    compute_multilingual_masked_perplexity_single, compute_multilingual_mt5_perplexity_single,
    extract_extended_metrics_from_logs,
)
from IPython.display import display
from tqdm.notebook import tqdm
from transformers import (
    logging,
    AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM,
    Trainer, TrainingArguments,
    EarlyStoppingCallback
)
from peft import LoraConfig, PrefixTuningConfig, get_peft_model, TaskType

# Set settings

In [2]:
tqdm.pandas()

In [3]:
# Suppress specific warnings from the transformers library
logging.set_verbosity_error()

# Common

In [4]:
# gpu device 
device = get_device()


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}
Using device: mps


In [5]:
# spt models
spt_models = {
    #"bpe": spm.SentencePieceProcessor("spt/spt_bpe.model"),
    "unigram": spm.SentencePieceProcessor("spt/spt_unigram.model"),
}

In [ ]:
train_agrs = {
    "warmup_steps": 500,
    "weight_decay": 0.01,
    "save_strategy": "epoch",
    "save_total_limit": 2,
    "fp16": False,
    "bf16": True,
    "eval_strategy": "epoch",
    "load_best_model_at_end": True,
    "metric_for_best_model": "loss",
    "greater_is_better": False,
    "logging_steps": 1000,
    "optim": "adamw_torch_fused",
    "auto_find_batch_size": True,
    "disable_tqdm": False,
}

# Function

In [9]:
def apply_lora(model, model_name, is_student):
    """
    Applies LoRA for efficient fine-tuning.
    """

    # Select correct LoRA target layers
    if "t5" in model_name.lower():
        target_modules = ["q", "v"]  # LoRA for T5/mT5
    else:
        target_modules = ["query", "value"]  # LoRA for BERT

    # Define LoRA Configuration
    if is_student:
        lora_config = LoraConfig(
            r=4,                    # Rank of LoRA matrices
            lora_alpha=8,           # Scaling factor
            target_modules=target_modules,  
            lora_dropout=0.05,      # Prevents overfitting
            bias="none"
        )
    else:
        lora_config = LoraConfig(
            r=8,                    # Rank of LoRA matrices
            lora_alpha=16,          # Scaling factor
            target_modules=target_modules,  
            lora_dropout=0.1,       # Prevents overfitting
        )

    # Apply LoRA
    model = get_peft_model(model, lora_config)

    # Move model to GPU
    model.to(device)

    print(f"LoRA applied to {model_name} (Target Modules: {target_modules})")
    
    return model

In [10]:
def apply_prefix_tuning(model):
    """
    Applies Prefix-Tuning to the model by adding tunable prefix parameters.
    """
    prefix_config = PrefixTuningConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,  # Required for mT5
        num_virtual_tokens=20,  # Number of prefix tokens to learn
        encoder_hidden_size=model.config.d_model,  # Same as mT5 hidden size
        prefix_projection=True  # Enables projection into model hidden state
    )

    model = get_peft_model(model, prefix_config)
    model.to(device)

    print("Prefix-Tuning applied with {} virtual tokens.".format(prefix_config.num_virtual_tokens))
    
    return model

# Data Preprocessing
Datasets used for training:
- myXNLI & ALT Corpus (normalized)
- Back-translated datasets (NLLB, Seamless M4T)
- Pseudo Parallel Corpus (Minilm, LaBse)

In [10]:
# Load and process dataset
def load_and_rename_columns_multilingual(file_name):
    df = load_gen_df(f"{file_name}")

    column_mapping = {
        "english": "source",
        "burmese": "target",
        "english_back_translated": "source",
        "burmese_translated": "target",
    }
    
    df = df.rename(columns=column_mapping)
    
    # Ensure only required columns exist
    df = df[["source", "target"]]

    return df

In [11]:
# datasets
datasets = {
    "normal": [
        "myxnli_normalized_1", 
        "myxnli_normalized_2", 
        "alt_combined_normalized"
    ],
    "nllb_back_translated": [
        "myxnli_nllb_back_translated_final_1", 
        "myxnli_nllb_back_translated_final_2", 
        "alt_combined_nllb_back_translated_final"
    ],
    "seamless_m4t_back_translated": [
        "myxnli_seamless_m4t_back_translated_final_1", 
        "myxnli_seamless_m4t_back_translated_final_2", 
        "alt_combined_seamless_m4t_back_translated_final"
    ],
    "minilm_pseudo_parallel_corpus": [
        "minilm_pseudo_parallel_corpus_final"
    ],
    "labse_pseudo_parallel_corpus": [
        "labse_pseudo_parallel_corpus_final"
    ],
}

In [12]:
# Load and process datasets
loaded_datasets = {}
for key, file_list in datasets.items():
    loaded_datasets[key] = [load_and_rename_columns_multilingual(file) for file in file_list]

In [13]:
# combine all datasets
combined = pd.concat(
    loaded_datasets["normal"] + 
    loaded_datasets["nllb_back_translated"] + 
    loaded_datasets["seamless_m4t_back_translated"],
    ignore_index=True
)

In [14]:
# Shuffle the data to prevent order bias
combined = combined.sample(frac=1).reset_index(drop=True)

In [15]:
# display combined dataset
display(combined.head())

,source,target
0,disheartened in hong kong,ဟောင်ကောင်မှာ စိတ်ပျက်သွားတယ်။
1,kyushu is an important part of japanese mythol...,Kyushu ဟာ ဂျပန် ဒဏ္ဍာရီရဲ့ အရေးပါတဲ့ အစိတ်အပို...
2,it's simple for sure.,ဒါက ရိုးရှင်းတဲ့အတွက် အသေအချာပါ။
3,and they think that i'm better than i am.,ပြီးတော့ သူတို့က ငါထက် ပိုကောင်းတယ်လို့ ထင်နေက...
4,"there were jokes about these comments, even th...",ဒီမှတ်ချက်တွေအကြောင်း နောက်ပြောင်တာတွေ ရှိခဲ့တ...


In [16]:
# print length
print(f"Combined dataset length: {len(combined)}")

Combined dataset length: 1627576


In [17]:
# save data
save_model_variants_df(combined, "combined")

In [18]:
# combine all datasets including pesudo parallel
combined_pseudo_parallel_corpus = pd.concat(
    loaded_datasets["normal"] + 
    loaded_datasets["nllb_back_translated"] + 
    loaded_datasets["seamless_m4t_back_translated"] +
    loaded_datasets["minilm_pseudo_parallel_corpus"] +
    loaded_datasets["labse_pseudo_parallel_corpus"],
    ignore_index=True
)

In [19]:
# Shuffle the data to prevent order bias
combined_pseudo_parallel_corpus = combined_pseudo_parallel_corpus.sample(frac=1).reset_index(drop=True)

In [20]:
# display combined dataset
display(combined_pseudo_parallel_corpus.head())

,source,target
0,i know you've never done it.,မင်းမလုပ်ဘူးဆိုတာ ငါသိတယ်။
1,people who shop at tiffany's are generally ver...,Tiffany's မှာ ဈေးဝယ်သူတွေဟာ အများအားဖြင့် အလွန...
2,daniel told his colleagues that they are to bl...,ဟောလိဝုဒ်၏ လက်ရှိအခြေအနေအတွက် ၎င်းတို့ကို အပြစ...
3,julia sweeney's little sister is dead.,Julia Sweeney ရဲ့ ညီမလေး က သေဆုံးသွားတယ်
4,she doesn't call this mission ambitious.,သူမဟာ ဒီတာဝန်ကို ရည်မှန်းချက်ကြီးတယ်လို့ မခေါ်...


In [21]:
# display data size
print(f"Total Size: {len(combined_pseudo_parallel_corpus)}")

Total Size: 3001668


In [22]:
# save data
save_model_variants_df(combined_pseudo_parallel_corpus, "combined_pseudo_parallel_corpus")

# 1. Fine-Tuning Transformer Models for Burmese
This notebook fine-tunes three transformer models:
- mBERT (best perplexity, but weak BLEU/ROUGE)
- mT5 (best for generation, but requires more data)
- XLM-R (good BLEU/ROUGE, but poor perplexity)

Apply:
- Sentence-Piece Tokenization for Burmese segmentation
- LoRA for efficient fine-tuning
- Prefix-Tuning for lightweight adaptations
- Mixed Precision Training for speed improvements

In [ ]:
# model names
train_model_names = {
    "mBERT": "bert-base-multilingual-cased",
    "mT5": "google/mt5-small",
    "XLM-R": "xlm-roberta-base"
}

In [ ]:
# train tokenizers
train_tokenizers = {
    "mBERT": AutoTokenizer.from_pretrained(train_model_names["mBERT"]),
    "mT5": AutoTokenizer.from_pretrained(train_model_names["mT5"], use_fast=False, legacy=True),
    "XLM-R": AutoTokenizer.from_pretrained(train_model_names["XLM-R"])
}

In [ ]:
# train models
train_models = {
    "mBERT": AutoModelForMaskedLM.from_pretrained(train_model_names["mBERT"]).to(device),
    "mT5": AutoModelForSeq2SeqLM.from_pretrained(train_model_names["mT5"]).to(device),
    "XLM-R": AutoModelForMaskedLM.from_pretrained(train_model_names["XLM-R"]).to(device)
}

## Tokenize

In [ ]:
def tokenize(examples, tokenizer, spt_tokenizer, model_name):
    """
    Tokenizes Burmese text using the selected SentencePiece model before applying Transformer tokenization.
    """
    # Apply SentencePiece Tokenization
    batch_source_tokens = [spt_tokenizer.encode(text, out_type=str) for text in examples["source"]]
    batch_target_tokens = [spt_tokenizer.encode(text, out_type=str) for text in examples["target"]]

    # Flatten and convert tokens to text
    batch_source_texts = [" ".join(tokens) for tokens in batch_source_tokens]
    batch_target_texts = [" ".join(tokens) for tokens in batch_target_tokens]

    # Flatten and convert tokens to text
    batch_source_texts = [" ".join(tokens) for tokens in batch_source_tokens]
    batch_target_texts = [" ".join(tokens) for tokens in batch_target_tokens]

    if "t5" in model_name.lower():
        # Tokenize using mT5 tokenizer (Seq2Seq)
        model_inputs = tokenizer(
            batch_source_texts, 
            padding="max_length", 
            truncation=True, 
            max_length=512
        )
        
        # Tokenize target (labels) separately (mT5 requires explicit target tokenization)
        labels = tokenizer(
            batch_target_texts, 
            padding="max_length", 
            truncation=True, 
            max_length=512
        )["input_ids"]
        
        # Replace pad tokens in labels with -100 (Hugging Face ignores -100 in loss calculation)
        labels = [[-100 if token == tokenizer.pad_token_id else token for token in label] for label in labels]

        model_inputs["labels"] = labels  # Assign labels for training

        return model_inputs
    
    else:
        # Tokenize using XLM-R tokenizer
        model_inputs = tokenizer(
            batch_source_texts, 
            padding="max_length", 
            truncation=True, 
            max_length=512
        )
        labels = tokenizer(
            batch_target_texts, 
            padding="max_length", 
            truncation=True, 
            max_length=512
        )["input_ids"]

        model_inputs["labels"] = labels  # Assign labels for MLM-style training

        return model_inputs

In [ ]:
# tokenize for each model and spt
for model_name, tokenizer in train_tokenizers.items():
    for spt_name, spt_tokenizer in spt_models.items():
        # load dataset
        if "t5" in model_name.lower():
            dataset = load_model_variants_df("combined_pseudo_parallel_corpus")
        else:
            dataset = load_model_variants_df("combined")

        # Split into 80% train, 20% test
        train_df = dataset.sample(frac=0.8, random_state=42)
        test_df = dataset.drop(train_df.index)

        # Convert to Hugging Face Dataset
        train_df = convert_to_hf(train_df)
        test_df = convert_to_hf(test_df)

        # apply tokenization to train dataset
        train_tokenized = train_df.map(
            lambda x, _: tokenize(x, tokenizer, spt_tokenizer, model_name),
            batched=True,
            batch_size=16,
            desc=f"Tokenizing train dataset for {model_name} with {spt_name}",
            with_indices=True,  # Passing index as a second argument
            num_proc=10
        )

        # memory occur when mT5 with unigram
        if "t5" in model_name.lower() and spt_name == "unigram":
            chunk_size = 5
        else:
            chunk_size = 1

        # save
        save_model_variants_hf(train_tokenized, f"{model_name.lower()}_{spt_name}_train", chunk_size)

        # apply tokenization to train dataset
        test_tokenized = test_df.map(
            lambda x, _: tokenize(x, tokenizer, spt_tokenizer, model_name),
            batched=True,
            batch_size=16,
            desc=f"Tokenizing test dataset for {model_name} with {spt_name}",
            with_indices=True,  # Passing index as a second argument
            num_proc=10
        )

        # save
        save_model_variants_hf(test_tokenized, f"{model_name.lower()}_{spt_name}_test", 1)

## Fine-Tuning

In [ ]:
# tokenized train dataset
tokenized_train_datasets = {
    model_name: {
        spt_name: load_model_variants_hf(f"{model_name.lower()}_{spt_name}_train")
        for spt_name in spt_models.keys()
    }
    for model_name in train_tokenizers.keys()
}

In [ ]:
# tokenized test dataset
tokenized_test_datasets = {
    model_name: {
        spt_name: load_model_variants_hf(f"{model_name.lower()}_{spt_name}_test")
        for spt_name in spt_models.keys()
    }
    for model_name in train_tokenizers.keys()
}

In [ ]:
def fine_tune_model(model_name, spt_name, batch_size):
    """
    Fine-tunes the model with LoRA on the specified SentencePiece tokenization (SPT).
    """
    print(f"Fine-tuning {model_name} using SPT-{spt_name.upper()}...")

    # Load tokenizer & model
    tokenizer = train_tokenizers[model_name]
    model = train_models[model_name]

    # Move model to GPU before applying LoRA
    model.to(device)

    # Apply LoRA for efficient parameter tuning
    model = apply_lora(model, model_name, False)

    if "t5" in model_name.lower():
        model = apply_prefix_tuning(model)

    # display trainable parameters
    print(model.print_trainable_parameters())
    
    # load dataset
    train_data = tokenized_train_datasets[model_name][spt_name]
    val_data = tokenized_test_datasets[model_name][spt_name]

    # for debug, remove comment
    #train_data = train_data.select(range(100))
    #val_data = val_data.select(range(100))

    trained_model_name = f"{model_name}_{spt_name.upper()}"

    # Define training arguments
    training_args = TrainingArguments(
        **train_agrs,
        output_dir=f"model-variants/results/{trained_model_name}",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=5,
        learning_rate=5e-5,
        logging_dir=f"model-variants/logs/{trained_model_name}",
        label_names=["labels"],
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=val_data,
        processing_class=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the model
    trainer.train()

    # Save trained model and tokenizer
    save_path = f"model-variants/models/{trained_model_name}"
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    print(f"Model `{model_name}` fine-tuned and saved at `{save_path}`.")

### mBert

#### BPE

In [ ]:
# fine tune with SPT-BPE
fine_tune_model("mBERT", "bpe", 16)

In [ ]:
# load train results
mbert_bpe_trained_results = extract_extended_metrics_from_logs("mBERT_BPE")

In [ ]:
# display train results
display(mbert_bpe_trained_results)

In [ ]:
# save train results
save_model_variants_gen_df(mbert_bpe_trained_results, "mbert_bpe_trained_results")

#### Unigram

In [ ]:
# fine tune with SPT-Unigram
fine_tune_model("mBERT", "unigram", 16)

In [ ]:
# load train results
mbert_unigram_trained_results = extract_extended_metrics_from_logs("mBERT_UNIGRAM")

In [ ]:
# display train results
display(mbert_unigram_trained_results)

In [ ]:
# save train results
save_model_variants_gen_df(mbert_unigram_trained_results, "mbert_unigram_trained_results")

### XLM-R

#### BPE

In [ ]:
# fine tune with SPT-BPE
fine_tune_model("XLM-R", "bpe", 8)

In [ ]:
# load train results
xlmr_bpe_trained_results = extract_extended_metrics_from_logs("XLM-R_BPE")

In [ ]:
# display train results
display(xlmr_bpe_trained_results)

In [ ]:
# save train results
save_model_variants_gen_df(xlmr_bpe_trained_results, "xlmr_bpe_trained_results")

#### Unigram

In [ ]:
# fine tune with SPT-Unigram
fine_tune_model("XLM-R", "unigram", 8)

In [ ]:
# load train results
xlmr_unigram_trained_results = extract_extended_metrics_from_logs("XLM-R_UNIGRAM")

In [ ]:
# display train results
display(xlmr_unigram_trained_results)

In [ ]:
# save train results
save_model_variants_gen_df(xlmr_unigram_trained_results, "xlmr_unigram_trained_results")

### mT5

#### BPE

In [ ]:
# fine tune with SPT-BPE
fine_tune_model("mT5", "bpe", 8)

In [ ]:
# load train results
mt5_bpe_trained_results = extract_extended_metrics_from_logs("mT5_BPE")

In [ ]:
# display train results
display(mt5_bpe_trained_results)

In [ ]:
# save train results
save_model_variants_gen_df(mt5_bpe_trained_results, "mt5_bpe_trained_results")

#### Unigram

In [ ]:
# fine tune with SPT-Unigram
fine_tune_model("mT5", "unigram", 8)

In [ ]:
# load train results
mt5_unigram_trained_results = extract_extended_metrics_from_logs("mT5_UNIGRAM")

In [ ]:
# display train results
display(mt5_unigram_trained_results)

In [ ]:
# save train results
save_model_variants_gen_df(mt5_unigram_trained_results, "mt5_unigram_trained_results")

## Generate Predictions

In [16]:
# Function to generate predictions for fine-tuned model using Hugging Face Dataset
def generate_predictions_fine_tuned_model(model_name, spt_name):
    base_model_name = train_model_names[model_name]

    # Load tokenizers & models
    model, tokenizer = get_fine_tuned_model(model_name, spt_name, base_model_name, device)
    model.eval()

    # Load dataset 
    dataset = load_models_df("multilingual_combined")

    dataset = convert_to_hf(dataset)

    # remove comment for debug
    dataset = dataset.select(range(10))

    # Run text generation
    if "t5" in model_name.lower():
        dataset = generate_mt5_predictions(model, tokenizer, device)
    else:
        dataset = generate_masked_predictions_hf(dataset, model, tokenizer, device)

    # Display results
    display(dataset.to_pandas().head())

    # Save dataset
    save_model_variants_df(dataset, f"{model_name}_{spt_name}_trained_predictions")

### mBERT

In [ ]:
# generate predictions for mbert with BPE
generate_predictions_fine_tuned_model("mBERT", "bpe")

In [ ]:
# generate predictions for mbert with Unigram
generate_predictions_fine_tuned_model("mBERT", "unigram")

### XLM-R

In [ ]:
# generate predictions for XLM-R with BPE
generate_predictions_fine_tuned_model("XLM-R", "bpe")

In [ ]:
# generate predictions for XLM-R with Unigram
generate_predictions_fine_tuned_model("XLM-R", "unigram")

### mT5

In [ ]:
# generate predictions for mT5 with BPE
generate_predictions_fine_tuned_model("mT5", "bpe")

In [ ]:
# generate predictions for mT5 with Unigram
generate_predictions_fine_tuned_model("mT5", "unigram")

## Evaluate Model Performance
Compute BLEU, ROUGE-1, ROUGE-2, ROUGE-3, ROUGE-L, chrF-S, BERTScore and Perplexity scores.

### Metrics

In [ ]:
# Function to Compute Metrics for Fine-Tuned Model using HF Dataset
def compute_metric_fine_tuned_model(model_name, spt_name):
    # Load dataset
    metrics_dataset = load_model_variants_df(f"{model_name}_{spt_name}_trained_predictions")

    # Compute metrics
    print(f"Processing Data for {model_name} with {spt_name.upper()}...")
    metrics_dataset = compute_metrics_hf(metrics_dataset)

    # Display results
    print(f"Metrics scores for {model_name} with {spt_name.upper()}:")
    print(f"BLEU Score: {metrics_dataset['bleu'].mean()}")
    print(f"ROUGE-1 Score: {metrics_dataset['rouge-1'].mean()}")
    print(f"ROUGE-2 Score: {metrics_dataset['rouge-2'].mean()}")
    print(f"ROUGE-L Score: {metrics_dataset['rouge-l'].mean()}")
    print(f"chrF-S Score: {metrics_dataset['chrf-s'].mean()}")
    print(f"BERT Score: {metrics_dataset['bert_score'].mean()}")

    # Save results
    save_tmp_df(metrics_dataset, f"{model_name}_{spt_name}_trained_metrics")

#### mBERT

In [ ]:
# compute metric for fine tuned mBERT with BPE
compute_metric_fine_tuned_model("mBERT", "bpe")

In [ ]:
# compute metric for fine tuned mBERT with Unigram
compute_metric_fine_tuned_model("mBERT", "unigram")

#### XLM-R

In [ ]:
# compute metric for fine tuned XLM-R with BPE
compute_metric_fine_tuned_model("XLM-R", "bpe")

In [ ]:
# compute metric for fine tuned XLM-R with Unigram
compute_metric_fine_tuned_model("XLM-R", "unigram")

#### mT5

In [ ]:
# compute metric for fine tuned mT5 with BPE
compute_metric_fine_tuned_model("mT5", "bpe")

In [ ]:
# compute metric for fine tuned mT5 with Unigram
compute_metric_fine_tuned_model("mT5", "unigram")

### Perplexity

In [ ]:
# Function to compute perplexity for fine-tuned model using HF Dataset
def compute_perplexity_fine_tuned_model(model_name, spt_name):
    # Load dataset as Hugging Face Dataset
    perplexity = load_model_variants_hf(f"{model_name}_{spt_name}_trained_predictions")

    # Load tokenizers & models
    model, tokenizer = get_fine_tuned_model(model_name, spt_name, device)
    model.to(device)
    model.eval()

    # Compute perplexity using HF Dataset
    if "t5" in model_name.lower():
        perplexity = perplexity.map(lambda example: {
            "perplexity": compute_multilingual_mt5_perplexity_single(example["generated"], model, tokenizer, device)
        })
    else:
        perplexity = perplexity.map(lambda example: {
            "perplexity": compute_multilingual_masked_perplexity_single(example["generated"], model, tokenizer, device)
        })

    # Display Perplexity Score
    print(f"Perplexity Score: {perplexity['perplexity'].mean()}")

    # Save dataset
    save_tmp_df(perplexity, f"{model_name}_{spt_name}_trained_perplexity")

#### mBERT

In [ ]:
# compute perplexity with BPE
compute_perplexity_fine_tuned_model("mBERT", "bpe")

In [ ]:
# compute perplexity with Unigram
compute_perplexity_fine_tuned_model("mBERT", "unigram")

#### XLM-R

In [ ]:
# compute perplexity with BPE
compute_perplexity_fine_tuned_model("XLM-R", "bpe")

In [ ]:
# compute perplexity with Unigram
compute_perplexity_fine_tuned_model("XLM-R", "unigram")

#### mt5

In [ ]:
# compute perplexity with BPE
compute_perplexity_fine_tuned_model("mT5", "bpe")

In [ ]:
# compute perplexity with Unigram
compute_perplexity_fine_tuned_model("mT5", "unigram")

### Save Evaluation Results

In [ ]:
# combine evaluation results
for model_name in train_model_names.keys():
    for spt_name in spt_models.keys():
        print(f"Processing {model_name}...")

        evaluation_results = load_model_variants_df(f"{model_name}_{spt_name}_trained_predictions")

        # load metrics and set
        metrics = load_tmp_df(f"{model_name}_{spt_name}_trained_metrics")
        evaluation_results["bleu"] = metrics["bleu"]
        evaluation_results["rouge-1"] = metrics["rouge-1"]
        evaluation_results["rouge-2"] = metrics["rouge-2"]
        evaluation_results["rouge-l"] = metrics["rouge-l"]
        evaluation_results["chrf-s"] = metrics["chrf-s"]
        evaluation_results["bert_score"] = metrics["bert_score"]

        # load perplexity and set
        perplexity = load_tmp_df(f"{model_name}_{spt_name}_perplexity")
        evaluation_results["perplexity"] = perplexity["perplexity"]

        save_model_variants_df(evaluation_results, f"{model_name}_{spt_name}_trained_evaluation_results")

## Benchmarking and Analysis
Compare the performance of LSTM BPE, LSTM Unigram, mBERT, and XLM-R using BLEU, ROUGE, chrF-S, BERT Score and Perplexity.

In [ ]:
# load data
trained_benchmarking_datasets = {}
for model_name in train_model_names.keys():
    for spt_name in spt_models.keys():
        df = load_model_variants_df(f"{model_name}_{spt_name}_trained_evaluation_results")
        trained_benchmarking_datasets[f"{model_name} {spt_name.upper()}"] = df

In [ ]:
# convert to mean score df
trained_benchmarking_mean_scores = convert_to_mean_scores_df(trained_benchmarking_datasets)

In [ ]:
# Display mean scores
display(trained_benchmarking_mean_scores)

In [ ]:
# save benchmarking results
save_model_variants_df(trained_benchmarking_mean_scores, "trained_evaluation_results")

# 2. Enhance Burmese Contextual Representations
- Use LASER, mUSE, and FastText for cross-lingual and morphology-aware training.
- Fine-tune mBERT, XLM-R on Burmese dataset after adding contextual embedded.
- Train models again using combined embedded.

In [ ]:
# embedding model names
embedding_model_names = ["mBERT", "XLM-R"]

In [ ]:
# tokenized train dataset
embedded_train_datasets = {
    model_name: {
        spt_name: load_model_variants_hf(f"contextual_embedded_{model_name.lower()}_{spt_name}_train")
        for spt_name in spt_models.keys()
    }
    for model_name in embedding_model_names
}

In [ ]:
# tokenized test dataset
embedded_test_datasets = {
    model_name: {
        spt_name: load_model_variants_hf(f"contextual_embedded_{model_name.lower()}_{spt_name}_test")
        for spt_name in spt_models.keys()
    }
    for model_name in embedding_model_names
}

## Tokenize

Moved to embeddings-dataset-preparation.ipynb

## Fine Tuning

In [ ]:
def fine_tune_with_contextual_embeddings(model_name, spt_name, batch_size):
    """
    Second-round fine-tuning for XLM-R and mBERT using LoRA + Contextual Embeddings.
    """
    print(f"Fine-tuning {model_name} using SPT-{spt_name.upper()} with contextual embeddings...")

    # Load tokenizer & model
    model, tokenizer = get_fine_tuned_model(model_name, spt_name, device)

    # Apply LoRA (Retaining Previous Fine-Tuning Weights)
    model = apply_lora(model, model_name, False)

    # Display Trainable Parameters
    print(model.print_trainable_parameters())

    # load dataset
    train_data = embedded_train_datasets[model_name][spt_name]
    val_data = embedded_test_datasets[model_name][spt_name]

    # Debugging: Use Smaller Dataset for Fast Testing (Uncomment if Needed)
    # train_data = train_data.select(range(100))
    # val_data = val_data.select(range(100))

    trained_model_name = f"Embedded_{model_name}_{spt_name.upper()}"

    # Define Training Arguments
    training_args = TrainingArguments(
        **train_agrs,
        output_dir=f"model-variants/results/{trained_model_name}",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=5,
        learning_rate=3e-5,
        logging_dir=f"model-variants/logs/{trained_model_name}",
        label_names=["labels", "input_ids", "contextual_embeds"],
    )

    # Initialize Trainer with Contextual Embeddings
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=val_data,
        processing_class=tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the Model
    trainer.train()

    # Save Trained Model and Tokenizer
    save_path = f"model-variants/models/{trained_model_name}"
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    print(f"Model `{model_name}` fine-tuned with contextual embeddings and saved at `{save_path}`.")

### mBERT

#### BPE

In [ ]:
# fine tune with SPT-BPE
fine_tune_with_contextual_embeddings("mBERT", "bpe", 16)

In [ ]:
# load train results
mbert_bpe_embedded_trained_results = extract_extended_metrics_from_logs("Embedded_mBERT_BPE")

In [ ]:
# display train results
display(mbert_bpe_embedded_trained_results)

In [ ]:
# save train results
save_model_variants_gen_df(mbert_bpe_embedded_trained_results, "mbert_bpe_embedded_trained_results")

#### Unigram

In [ ]:
# fine tune with SPT-Unigram
fine_tune_with_contextual_embeddings("mBERT", "unigram", 16)

In [ ]:
# load train results
mbert_unigram_embedded_trained_results = extract_extended_metrics_from_logs("Embedded_mBERT_UNIGRAM")

In [ ]:
# display train results
display(mbert_unigram_embedded_trained_results)

In [ ]:
# save train results
save_model_variants_gen_df(mbert_unigram_embedded_trained_results, "mbert_unigram_embedded_trained_results")

### XLM-R

#### BPE

In [ ]:
# fine tune with SPT-BPE
fine_tune_with_contextual_embeddings("XLM-R", "bpe", 8)

In [ ]:
# load train results
xlmr_bpe_embedded_trained_results = extract_extended_metrics_from_logs("Embedded_XLM-R_BPE")

In [ ]:
# display train results
display(xlmr_bpe_embedded_trained_results)

In [ ]:
# save train results
save_model_variants_gen_df(xlmr_bpe_embedded_trained_results, "xlmr_bpe_embedded_trained_results")

#### Unigram

In [ ]:
# fine tune with SPT-Unigram
fine_tune_with_contextual_embeddings("XLM-R", "unigram", 8)

In [ ]:
# load train results
xlmr_unigram_embedded_trained_results = extract_extended_metrics_from_logs("Embedded_XLM-R_UNIGRAM")

In [ ]:
# display train results
display(xlmr_unigram_embedded_trained_results)

In [ ]:
# save train results
save_model_variants_gen_df(xlmr_unigram_embedded_trained_results, "xlmr_unigram_embedded_trained_results")

## Generate Predictions

In [ ]:
# Function to generate predictions for embedded fine-tuned model using Hugging Face Dataset
def generate_predictions_embedded_fine_tuned_model(model_name, spt_name):
    # Load tokenizers & models
    model, tokenizer = get_embedded_fine_tuned_model(model_name, spt_name, device)

    # Load dataset 
    tokenized_dataset = load_model_variants_hf(f"{model_name}_{spt_name}")

    # remove comment for debug
    #tokenized_dataset = tokenized_dataset.select(range(10))

    # Run text generation
    tokenized_dataset = generate_masked_predictions_hf(tokenized_dataset, model, tokenizer, device)

    # remove columns
    tokenized_dataset = tokenized_dataset.select_columns(["source", "target", "generated"])

    # Display results
    display(tokenized_dataset.to_pandas().head())

    # Save dataset
    save_model_variants_df(tokenized_dataset, f"model-variants/{model_name}_{spt_name}_embedded_predictions")

### mBERT

In [ ]:
# generate predictions for mbert with BPE
generate_predictions_embedded_fine_tuned_model("mBERT", "bpe")

In [ ]:
# generate predictions for mbert with Unigram
generate_predictions_embedded_fine_tuned_model("mBERT", "unigram")

### XLM-R

In [ ]:
# generate predictions for XLM-R with BPE
generate_predictions_embedded_fine_tuned_model("XLM-R", "bpe")

In [ ]:
# generate predictions for XLM-R with Unigram
generate_predictions_embedded_fine_tuned_model("XLM-R", "unigram")

## Evaluate Model Performance
Compute BLEU, ROUGE-1, ROUGE-2, ROUGE-3, ROUGE-L, chrF-S, BERTScore and Perplexity scores.

### Metrics

In [ ]:
# Function to Compute Metrics for embedded Fine-Tuned Model using HF Dataset
def compute_metric_embedded_fine_tuned_model(model_name, spt_name):
    # Load dataset
    metrics_dataset = load_model_variants_df(f"{model_name}_{spt_name}_embedded_predictions")

    # Compute metrics
    print(f"Processing Data for {model_name} with {spt_name.upper()}...")
    metrics_dataset = compute_metrics_hf(metrics_dataset)

    # Display results
    print(f"Metrics scores for {model_name} with {spt_name.upper()}:")
    print(f"BLEU Score: {metrics_dataset['bleu'].mean()}")
    print(f"ROUGE-1 Score: {metrics_dataset['rouge-1'].mean()}")
    print(f"ROUGE-2 Score: {metrics_dataset['rouge-2'].mean()}")
    print(f"ROUGE-L Score: {metrics_dataset['rouge-l'].mean()}")
    print(f"chrF-S Score: {metrics_dataset['chrf-s'].mean()}")
    print(f"BERT Score: {metrics_dataset['bert_score'].mean()}")

    # Save results
    save_tmp_df(metrics_dataset, f"{model_name}_{spt_name}_embedded_metrics")

#### mBERT

In [ ]:
# compute metric for fine tuned mBERT with BPE
compute_metric_embedded_fine_tuned_model("mBERT", "bpe")

In [ ]:
# compute metric for fine tuned mBERT with Unigram
compute_metric_embedded_fine_tuned_model("mBERT", "unigram")

#### XLM-R

In [ ]:
# compute metric for fine tuned XLM-R with BPE
compute_metric_embedded_fine_tuned_model("XLM-R", "bpe")

In [ ]:
# compute metric for fine tuned XLM-R with Unigram
compute_metric_embedded_fine_tuned_model("XLM-R", "unigram")

### Perplexity

In [ ]:
# Function to compute perplexity for embedded fine-tuned model using HF Dataset
def compute_perplexity_embedded_fine_tuned_model(model_name, spt_name):
    # Load dataset as Hugging Face Dataset
    perplexity = load_model_variants_hf(f"{model_name}_{spt_name}_embedded_predictions")

    # Load tokenizers & models
    model, tokenizer = get_embedded_fine_tuned_model(model_name, spt_name, device)
    model.to(device)
    model.eval()

    # Compute perplexity using HF Dataset
    perplexity = perplexity.map(lambda example: {
        "perplexity": compute_multilingual_masked_perplexity_single(example["generated"], model, tokenizer, device)
    })

    # Display Perplexity Score
    print(f"Perplexity Score: {perplexity['perplexity'].mean()}")

    # Save dataset
    save_tmp_df(perplexity, f"{model_name}_{spt_name}_embedded_perplexity")

#### mBERT

In [ ]:
# compute perplexity with BPE
compute_perplexity_embedded_fine_tuned_model("mBERT", "bpe")

In [ ]:
# compute perplexity with Unigram
compute_perplexity_embedded_fine_tuned_model("mBERT", "unigram")

#### XLM-R

In [ ]:
# compute perplexity with BPE
compute_perplexity_embedded_fine_tuned_model("XLM-R", "bpe")

In [ ]:
# compute perplexity with Unigram
compute_perplexity_embedded_fine_tuned_model("XLM-R", "unigram")

### Save Evaluation Results

In [ ]:
# combine evaluation results
for model_name in embedding_model_names:
    for spt_name in spt_models.keys():
        print(f"Processing {model_name}...")

        evaluation_results = load_model_variants_df(f"{model_name}_{spt_name}_embedded_predictions")

        # load metrics and set
        metrics = load_tmp_df(f"{model_name}_{spt_name}_metrics")
        evaluation_results["bleu"] = metrics["bleu"]
        evaluation_results["rouge-1"] = metrics["rouge-1"]
        evaluation_results["rouge-2"] = metrics["rouge-2"]
        evaluation_results["rouge-l"] = metrics["rouge-l"]
        evaluation_results["chrf-s"] = metrics["chrf-s"]
        evaluation_results["bert_score"] = metrics["bert_score"]

        # load perplexity and set
        perplexity = load_tmp_df(f"{model_name}_{spt_name}_perplexity")
        evaluation_results["perplexity"] = perplexity["perplexity"]

        save_model_variants_df(evaluation_results, f"{model_name}_{spt_name}_embedded_evaluation_results")

## Benchmarking and Analysis
Compare the performance of LSTM BPE, LSTM Unigram, mBERT, and XLM-R using BLEU, ROUGE, chrF-S, BERT Score and Perplexity.

In [ ]:
# load data
embedded_benchmarking_datasets = {}
for model_name in embedding_model_names:
    for spt_name in spt_models.keys():
        df = load_model_variants_df(f"{model_name}_{spt_name}_trained_evaluation_results")
        embedded_benchmarking_datasets[f"{model_name} {spt_name.upper()}"] = df

In [ ]:
# convert to mean score df
embedded_benchmarking_mean_scores = convert_to_mean_scores_df(embedded_benchmarking_datasets)

In [ ]:
# Display mean scores
display(embedded_benchmarking_mean_scores)

In [ ]:
# save benchmarking results
save_model_variants_df(embedded_benchmarking_mean_scores, "embedded_evaluation_results")